## Train/Validation set 나누기


In [0]:
#train 1601458
#test start from 1601458

train =new_df[new_df['transaction_real_price']!=0]

# target = target.reset_index(drop=True) # 종속변수도 똑같이 진행 
train['weight'] = np.ones(len(train),dtype=np.int8) # 가중치를 1로 두는 열을 하나 만든다. 

XY_all = train
mask = np.random.rand(len(train)) < 0.8

XY_train = train[mask] # 훈련 0.8
XY_validate= train[~mask] # 검증셋 0.2 


In [0]:
train_seoul = train[train['city'] == 'seoul']
train_busan = train[train['city'] == 'busan']

In [0]:
#train_xy['transaction_real_price'] # train_y
#train_xy # ()columns = features)
test_df=new_df[new_df['transaction_real_price']==0]
test_df.info()
test_df["weight"] = np.ones(len(test_df), dtype=np.int8)

# test_df["y"] = new_df['transaction_real_price']

test_seoul = test[test['city'] == 'seoul']
test_busan = test[test['city'] == 'busan']

In [0]:
features2
# city 
# transaction_year_month
# transaction_date
# floor
# address_by_law
# district
# district_group
# above_49

# del features2[-1]
features2.append('city')
features2.append('transaction_year_month')
features2.append('transaction_date')
features2.append('floor')
features2.append('address_by_law')
features2.append('district')
features2.append('district_group')
features2.append('above_49')

### LightGBM

In [0]:
features2
XY_train

In [0]:
# Set parameter
param = {'task' : 'train',
         'eta' : 0.1,
    'objective' : 'regression',
    'boosting_type' : 'gbdt',
    'metric' : 'rmse',
    'min_data_in_leaf' : 16,
    'bagging_fraction' : 1,
    'bagging_freq' : 500,
    'max_bin' : 255,
    'num_leaves' : 107,
    'seed' : 48,} 

In [0]:
import lightgbm as lgbm 

# X_t=XY_train.loc[:,'year_of_completion':'Above_49'] 
# features_t = X_t.columns
# X_v=XY_validate.loc[:,'year_of_completion':'Above_49'] 
# features_v = X_v.columns

train2 = lgbm.Dataset(XY_train[list(features2)], label=XY_train['transaction_real_price'],feature_name=features2)

validate2 = lgbm.Dataset(XY_validate[list(features2)], label=XY_validate['transaction_real_price'],
                                 feature_name=features2, reference = train2)

# evalist = [(train2,'train'),(validate2,'eval')]
model = lgbm.train(param, train2, num_boost_round = 4000, valid_sets =validate2, early_stopping_rounds=10) 
# callback 함수(얼리스토핑)를 통해 과대적합이 일어나자마자 바로 학습을 종료하도록 구성. 
best_iteration = model.best_iteration # 가장 좋은 반복횟수를 저장,


In [0]:
# gridsearch_params = [
#    (num_leaves, min_data_in_leaf)
#    for num_leaves in range(105, 110) # 20, 21, 22, 23
#    for min_data_in_leaf in range(15, 20)] # 5, 6, 7, 8 


In [0]:
# lightGBM.cv 
# 교차 검증. 다양한 param들의 수정을 통해 cv의 성능을 확인한다. 

param = {'task' : 'train',
         'eta' : 0.1,
    'objective' : 'regression',
    'boosting_type' : 'gbdt',
    'metric' : 'rmse',
#     'min_data_in_leaf' : min_data_in_leaf,
    'bagging_fraction' : 0.85,
    'bagging_freq' : 100,
    'max_bin' : 255,
#     'num_leaves' : num_leaves,
    'seed' : 48,} 

# define initial best params ans rmse 

min_rmse = float("inf")
best_params = None
# xgboost의 교차검증 기능. 다양한 param들의 수정을 통해 cv의 성능을 확인한다. 
for num_leaves, min_data_in_leaf in gridsearch_params:
    print("CV with num_leaves={}, min_data_in_leaf={}".format(
                             num_leaves,
                             min_data_in_leaf))
    # Update our parameters
    param['num_leaves'] = num_leaves
    param['min_data_in_leaf'] = min_data_in_leaf
    
    # Run CV

    all_train = lgbm.Dataset(XY_all[list(features2)], label=XY_all['transaction_real_price'],weight=XY_all['weight'], feature_name=features2)

    cv_results = lgbm.cv(
            param,
            all_train,
            num_boost_round=4000,
            nfold=4,
            metrics='rmse',
            early_stopping_rounds=15,
            stratified=False,
            verbose_eval = 50,
            )
    
    # Update best score
    mean_rmse = min(cv_results['rmse-mean'])
    print('\tRMSE {}'.format(mean_rmse))
    if mean_rmse < min_rmse:
      min_rmse = mean_rmse
      best_params = (num_leaves,  min_data_in_leaf)

print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

In [0]:
# 전체 훈련 데이터에는 늘어난 양만큼 트리 개수를 늘린다. 
best_iteration = int(best_iteration * len(XY_all) / len(XY_train)) 
# 전체 훈련 데이터에 대한 LightGBM 전용 데이터를 생성한다. 
all_train = lgbm.Dataset(XY_all[list(features2)], label=XY_all['transaction_real_price'],weight=XY_all['weight'], feature_name=features2)
# LightGBM 모델 학습 
model2 = lgbm.train(param,all_train,num_boost_round= best_iteration)

# 변수 중요도를 출력한다.
# print('Feature Importance by split:')
# for kv in sorted([(k,v) for k,v in zip(features2, model.feature_importance("split"))],
#                 key=lambda kv:kv[1], reverse=True):
#   print(kv)

# print('Feature Importance by gain:')
# for kv in sorted([(k,v) for k,v in zip(features2, model.feature_importance("gain"))],
#                 key=lambda kv:kv[1], reverse=True):
#   print(kv)
  


In [0]:
# 테스트 데이터에 대한 예측 결과물을 리턴 
pred_test_lgbm = model2.predict(test_df[list(features2)],num_iteration=best_iteration,verbose = 1)
pred_test_lgbm

In [0]:
# 곱셈 후 , 제곱근을 구하는 방식으로 앙상블을 수행한다 
Y_test = np.sqrt(np.multiply(preds_test_xgb, pred_test_lgbm)) # 기하평균 
# Y_test = ((0.5*preds_test_xgb)+(0.5*pred_test_lgbm)) # 단순평균 
submission 

# 현재는 단일 모델을 쓰기 때문에 앙상블 코드를 쓰지 않는다. 
# XGBoost 모델은 생략.

In [0]:
df_submission = pd.read_csv('submission.csv')
df_test_predict  = pd.DataFrame({'key':df_submission.key, 'transaction_real_price':pred_test_lgbm})


df_test_predict.to_csv('submission_0219_1.csv',index=False) 
# XGBoost and lightGBM Ensemble